# GPT-2 Model

We are using HuggingFace's model! I am currently using [this article](https://www.modeldifferently.com/en/2021/12/generaci%C3%B3n-de-fake-news-con-gpt-2/) and [this HuggingFace link](https://huggingface.co/gpt2).

In [1]:
# uncomment for colab
# !pip install transformers datasets accelerate nvidia-ml-py3 evaluate

# import hugging face
from transformers import GPT2Tokenizer, GPT2LMHeadModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 

In [2]:
# create smaller dataset from our subset data
from datasets import Dataset
import pandas as pd
filename = 'data/5000_booksummaries.zip'
tokens_df = pd.read_csv(filename)
tokens_df.head(5)

,Text
0,Generate a book summary with genres Science Fi...
1,Generate a book summary with genres Fantasy:\n...
2,Generate a book summary with genres Crime Fict...
3,"Generate a book summary with genres Fiction, N..."
4,"Generate a book summary with genres War novel,..."


In [4]:
# split data into train and test/eval data
from sklearn.model_selection import train_test_split

# split into train (80%), val (10%), test (10%)
train_data, test_eval_dataset = train_test_split(tokens_df, test_size=0.2, random_state=8)
eval_set, test_set = train_test_split(test_eval_dataset, test_size=0.5, random_state=8)

# create HuggingFace Datasets
train_ds = Dataset.from_pandas(train_data)
eval_ds = Dataset.from_pandas(eval_set)
test_ds = Dataset.from_pandas(test_set)

In [5]:
# tokenize datasets
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["Text"], truncation=True)

train_tok_ds = train_ds.map(tokenize_function, batched=True).shuffle(seed=42)
eval_tok_ds = eval_ds.map(tokenize_function, batched=True).shuffle(seed=42)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained('gpt2')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
        output_dir="temp_model",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        evaluation_strategy='steps',
        per_device_train_batch_size=4,
        num_train_epochs=5,   # can change between 3 and 5
        save_total_limit=1,
        gradient_accumulation_steps=4, 
        gradient_checkpointing=True, 
        fp16=True) # using gradient accumulation and checkpointing to not take as much memory

trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_tok_ds,
        eval_dataset=eval_tok_ds,
    )

In [7]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
500,3.413300,3.311373
1000,3.269800,3.308220


TrainOutput(global_step=1250, training_loss=3.3184234375, metrics={'train_runtime': 3762.2058, 'train_samples_per_second': 5.316, 'train_steps_per_second': 0.332, 'total_flos': 8630108375040000.0, 'train_loss': 3.3184234375, 'epoch': 5.0})

In [8]:
# save local version
checkpoint = "./model_config"
model.save_pretrained(checkpoint)
tokenizer.save_pretrained(checkpoint)

('./model_config/tokenizer_config.json',
 './model_config/special_tokens_map.json',
 './model_config/vocab.json',
 './model_config/merges.txt',
 './model_config/added_tokens.json')

In [9]:
!zip -r /content/model_setup_5000_3.zip /content/model_config

  adding: content/model_config/ (stored 0%)
  adding: content/model_config/config.json (deflated 51%)
  adding: content/model_config/generation_config.json (deflated 24%)
  adding: content/model_config/merges.txt (deflated 53%)
  adding: content/model_config/vocab.json (deflated 68%)
  adding: content/model_config/special_tokens_map.json (deflated 74%)
  adding: content/model_config/pytorch_model.bin (deflated 9%)
  adding: content/model_config/tokenizer_config.json (deflated 70%)


# Testing the Model

Now, we can utilize our trained model parameters to conduct analysis and test on our dataset!

In [10]:
# load into model and tokenizer
model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)

In [16]:
# load input prompt
input_prompt = "Generate a book summary with genre science fiction:\n"
inputs = tokenizer(input_prompt, return_tensors="pt")

# generate output from pretrained experiments (see baseline file)
outputs = model.generate(**inputs, 
    max_length=150, 
    num_beams=2, 
    no_repeat_ngram_size=2, 
    do_sample=True,
    early_stopping=True)

# decode output and print out summary
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generate a book summary with genre science fiction:
 The story is set in the fictional universe of the post-apocalyptic future of Earth, and follows the development of a team of scientists who work on a new type of particle accelerator. The team is led by Dr. James Kirk, a highly respected physicist who has been working on particle accelerators since the 1970s. Kirk and his team have been trying to develop a novel accelerator that would allow them to explore the origins of life on Earth. However, Kirk's team has not succeeded in finding the right technology, so they are forced to use a series of bizarre and dangerous methods to try and create a device that could potentially revolutionize the world as we know it. In the end, the


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# use BERTScores to analyze
# !pip install bert_score
from evaluate import load
bertscore = load("bertscore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 7.5 MB/s eta 0:00:00


In [18]:
# truncate the test set data
"""def truncate_text(text):
  # keep only first words
  first_30_words = text.split(' ')[:30]
  return ' '.join(first_30_words)

trunc_test_ds = Dataset.from_pandas(test_set)
trunc_test_ds.map(truncate_text)"""

# run model to generate predictions

predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
print(results)

{'precision': [1.0, 0.9999999403953552], 'recall': [1.0, 0.9999999403953552], 'f1': [1.0, 0.9999999403953552], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.27.1)'}
